In [41]:
#from CS_StockData import *
#from CS_StockData import getStockData
#from CS_Graphs import graph_normalized_all
#from CS_Graphs import graph_normalized_single

#does not work for some reason (?)
#from CS_Graphs import graph_normalized_single

#############################################################################################
#Request ticker information
#tickers = input("Please provide stock symbols separated by commas, max 5:") #aapl (Apple, inc.)
#tickers = tickers.split(",")

#clean up white space and make all tickers upper case
#for x in range(0,len(tickers)):
#    tickers[x] = tickers[x].strip()
#    tickers[x] = tickers[x].upper()

#############################################################################################


#stock_names, stock_data = getStockData(tickers)



In [6]:
#cleaned up presentation of actual main program (no imports)

tickers = input("Please provide stock symbols separated by commas, max 5:") #aapl (Apple, inc.)
x, y = getStockData(tickers)

In [7]:
x

{'V': 'Visa Inc.', 'MSFT': 'Microsoft Corporation', 'AAPL': 'Apple Inc.'}

## Web Scraping / Data Gathering Module

In [23]:
#%%writefile CS_StockData.py

import requests
from bs4 import BeautifulSoup
from time import sleep
import csv 
import pandas as pd
import numpy as np
import re

def getStockData(user_input):
    '''Pulls data from Yahoo Finance'''

    tickers = cleanUserStockInput(user_input)
    
    stockData = {}
    stockNames = {}
    
    for stock in tickers:
        try:
            stock_url = "https://finance.yahoo.com/quote/"+stock+"/history?p="+stock+""
            r = requests.get(stock_url)
            data = r.text
            soup = BeautifulSoup(data)

            contents = []

            for row in soup.find_all('tbody'):    
                for srow in row.find_all('tr'):
                    for content in srow.find_all('td'):
                        if "Dividend" in content.text:
                            #removes prior entry of date of dividend data point and ignores dividend data point
                            contents.pop() 
                            continue
                        else:
                            contents.append(content.text)
            
            stockData[stock] = defineStockData(contents)
            stockNames[stock] = defineStockName(stock)

        except:
            continue
            #stockErrors.append(stock)
            
        #delay added to avoid triggering web scrapping blocks
        delay = [0.1, 0.5, 1, 2, 2.5]
        delay = np.random.choice(delay)

        sleep(delay)
    
    return stockNames, stockData


def defineStockData(raw_list):
    '''Transforming scraped stock data into columns of data'''

    #the 7 columns of data, decided not to include most in this analysis
    date=[]
    open_price=[]
    high_price=[]
    low_price=[]
    close_price=[]
    adj_close_price=[]
    volume = []
    percent_change = []
    normalized_returns = []

    #pulled all data into one long list, but there are 7 columns of data
    #list comprehension to break out the full list in groupings by the 7 columns
    adj_contents = [raw_list[i * 7:(i + 1) * 7] for i in range((len(raw_list) + 7 - 1) // 7 )]

    #assigning the values to the columns
    for i in range(0, len(adj_contents)): #change to a count for amount of days requested...up to 97
        date.append(adj_contents[i][0])
        open_price.append(float(adj_contents[i][1]))
        high_price.append(float(adj_contents[i][2]))
        low_price.append(float(adj_contents[i][3]))
        close_price.append(float(adj_contents[i][4]))
        adj_close_price.append(float(adj_contents[i][5].replace(",","")))
        volume.append(float(adj_contents[i][6].replace(",","")))

    #calculating the percent change from prior close to today        
    for j in range(0, len(adj_close_price)):
        if j == len(adj_close_price)-1:
            percent_change.append(0.00)
        else:
            percent_change.append(round(adj_close_price[j]/adj_close_price[j+1]-1,3))

    #normalizing returns for comparison across companies
    normalizer = 100

    for x in range(len(percent_change), 0,-1):
        if x == len(percent_change):
            normalized_returns.append(normalizer)
        else:
            normalizer = round(normalizer * (1+percent_change[x-1]),3)
            normalized_returns.append(normalizer)

    normalized_returns.reverse()

    return addStockToDF(date, open_price, high_price, low_price, close_price, adj_close_price, volume, percent_change,normalized_returns)



def addStockToDF(date,open_price,high_price,low_price,close_price,adj_close_price,volume, percent_change,normalized_returns):
    '''Placing stock data into panda data frame'''
    #placing the values into a panda data frame
    x = pd.DataFrame({"Date": date,
                      "Open Price": open_price,
                      "High Price": high_price,
                      "Low Price": low_price,
                      "Close Price": close_price,
                      "Adj Close Price": adj_close_price,
                      "Volume":volume,
                      "Percent Change": percent_change,
                      "Normalized Returns": normalized_returns})
    x["Date"] = pd.to_datetime(x["Date"]) #allows interpretation as dates vs. ordered numbers

    #placing data frame into dictionary with stock ticker as key
    return x



def cleanUserStockInput(tickers):
    '''Cleans and formats user stock input'''
    tickers = tickers.split(",")

    #clean up white space and make all tickers upper case
    for x in range(0,len(tickers)):
        tickers[x] = tickers[x].strip()
        tickers[x] = tickers[x].upper()

    return tickers
    

def defineStockName(ticker):
    '''Matches ticker with company name from CSV file'''
    
    #list obtained from https://dumbstockapi.com/
    stockList = pd.read_csv("StockInformationList.csv")
        
    for y in range(0, len(stockList)):
        if ticker == stockList["ticker"][y]:
            return stockList["name"][y]     


def pullStockName(df):
    '''To provide stock name given data frame format'''

    #format = df["Ticker"]
    #df = str(df)
    return re.findall(r"(.*?)", str(df))


def pullStockInformationList():
    '''Downloads CSV file of Ticker:Copmany Name'''
    # Question whether we want this as an option(?)


##########################################################
#GoogleTrends Data - pytrends [?]




## Data Processing Module

In [4]:
import pandas as pd
import numpy as np
from math import sqrt


##########################################################
# Moving Average Convergence-Divergence


def calcMACD(df,ma_fast,ma_slow,macd_period):
    '''Moving Average Convergence Divergence (MACD)'''
    #ma_fast is periods included in fast moving average
    #ma_slow is periods included in fast moving average
    #macd_period is periods included in MA of MA

    ma_fast = calcMovingAverage(df,ma_fast)
    ma_slow = calcMovingAverage(df,ma_slow)
    ma_diff = []
    macd = []

    for x in range(0,len(ma_slow)):
        ma_diff.append(ma_fast[x]-ma_slow[x])
    
    macd = calcMovingAverage(ma_diff,macd_period)

    #lists of different lengths
    return ma_diff, macd


def calcMovingAverage(df, a):
    '''General moving average function'''
    length = len(df)-a
    ma_data =[]

    for x in range(0,length):
        total = 0
        for y in range(0, a):
            total += df[x+y]
        ma_data.append(total/a)

    return ma_data


def pullMACDPoints(ma_diff,macd):
    '''Identify index position of where MACD lines cross'''
    #initiate list to capture points where lines cross
    indexMACDpoints = []

    #running prior period value to determine if new value indicates lines crossed
    prior_period = 0

    for x in range(0, len(macd)):
        
        if x == 0:
            prior_period = ma_diff[x]-macd[x]
        
        else:
            current_period = ma_diff[x]-macd[x]

            if current_period < 0 and prior_period >= 0:
                indexMACDpoints.append([x,"buy"])
                prior_period = current_period
            elif current_period > 0 and prior_period <= 0:
                indexMACDpoints.append([x,"sell"])
                prior_period = current_period
            else:
                continue
    
    return indexMACDpoints


##########################################################
# Volume-Price Trend


def calcVolPriceTrend(df):
    '''Calculates volume-price trend for stock'''
    #https://www.daytrading.com/volume-price-trend
    #used to understand if movements in price are supported by significant volume
    
    vpt = []
    day = []
    daily_change = 0
    daily_vpt = 0

    for x in range(0,len(df)-1):
        daily_change = (df["Adj Close Price"][x+1]-df["Adj Close Price"][x])/df["Adj Close Price"][x]
        daily_vpt = daily_vpt + df["Volume"][x+1]*daily_change              
        vpt.append(daily_vpt)
        day.append(df["Date"][x+1])
    
    return day, vpt


##########################################################
# Descriptive Statistics


def pullMaxValue(df):
    '''Returns maximum stock price over period - may be intraday'''
    
    value = df["High Price"].max()
    
    for x in range(0, len(df)):
        if value == df["High Price"][x]:
            date = df["Date"][x]
    
    return [value, date]


def pullMinValue(df):
    '''Returns minimum stock price over period - may be intraday'''
    
    value = df["Low Price"].min()
    
    for x in range(0, len(df)):
        if value == df["Low Price"][x]:
            date = df["Date"][x]
    
    return [value, date]


def calcVolatility(df):
    '''Returns the stock volatility over the period'''

    #based on adjusted close price

    mean = df["Adj Close Price"].mean()
    n = df["Adj Close Price"].count()-1
    volatility = 0

    for x in range(0,len(df)):
        volatility += (mean-df["Adj Close Price"][x])**2

    volatility = round(volatility/n,2) 

    return volatility


def calcStandardDeviation(df):
    '''Calculates the standard deviation over the period'''
    
    std = round(sqrt(calcVolatility(df)),2)

    return std


def calcPeriodReturn(df, beg, end):
    '''Returns closing share price for the period'''
    
    beginning_price = df["Open Price"][beg]
    closing_price = df["Adj Close Price"][end]

    period_return = round((closing_price - beginning_price)/beginning_price,2)

    return period_return


def calcDescriptiveStats(df):
    return 


def calcComparativeStats(df):
    return 



In [13]:
print(y["V"]["Open Price"][0])
print(y["V"]["Adj Close Price"][5])
print(calcPeriodReturn(y["V"],0,5))

176.27
157.39
-0.11


In [220]:
#adjusted close price of Visa
basicLineGraph(y["V"]["Date"],y["V"]["Adj Close Price"])

In [218]:
#volume bar chart for Visa
basicBarGraph(y["V"]["Date"],y["V"]["Volume"])

In [219]:
#volume-price adjusted chart for Visa
#exaggerates price movements based on corresponding daily volume
a, b = calcVolPriceTrend(y["V"])
basicLineGraph(a,b)

In [18]:
graphMACD_buy_sell(y,"V",10,26,9)

In [11]:
showTableData(y["V"])

## Graphing Module

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


def basicLineGraph(x, y):
    '''Basic line graph for testing'''
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x,y=y, mode='lines'))
    fig.show()


def basicBarGraph(x, y):
    '''Basic bar graph for testing'''
    fig = go.Figure()
    fig.add_trace(go.Bar(x=x,y=y))
    fig.show()


def showTableData(df):
    '''Table displaying all columns of stock data'''

    ###Should clean up Date to remove all of the extra data beyond YYYY-MM-DD###

    fig = go.Figure(data=[go.Table(header=dict(values=list(df.columns),
                align='left'),
                cells=dict(values=[df["Date"],df["Open Price"],df["High Price"],df["Low Price"],df["Close Price"],df["Adj Close Price"],df["Volume"],df["Percent Change"],df["Normalized Returns"]],
                align='left'))
                ])

    fig.update_layout(autosize=False, width=1200,height=500, margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ))

    fig.update_layout(title='Data Table of Company X')

    fig.show()


def graphMACD_buy_sell(df, ticker, fast, slow, macd):
    '''Simplifying MACD graph call by ticker'''

    ma_fast, ma_slow = calcMACD(df[ticker]["Adj Close Price"], fast, slow, macd)
    macd_index = pullMACDPoints(ma_fast, ma_slow)
    x_axis = []

    for x in range(0,len(ma_slow)):
        x_axis.append(df[ticker]["Date"][x])
   
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_axis, y=ma_fast,
                        mode='lines',
                        name='fast MA'))
    fig.add_trace(go.Scatter(x=x_axis, y=ma_slow,
                        mode='lines',
                        name='slow MA'))

    for x in range(0, len(macd_index)):
        fig.add_annotation(
                    x=x_axis[macd_index[x][0]],
                    y=ma_slow[macd_index[x][0]],
                    text=macd_index[x][1])

    fig.update_annotations(dict(
                xref="x",
                yref="y",
                showarrow=True,
                arrowhead=7,
                ax=0,
                ay=-40
    ))

    fig.update_layout(title='MACD of {}'.format(defineStockName(ticker)))

    fig.show()


In [20]:
y["V"].columns

Index(['Date', 'Open Price', 'High Price', 'Low Price', 'Close Price',
       'Adj Close Price', 'Volume', 'Percent Change', 'Normalized Returns'],
      dtype='object')

In [35]:
a = []

for x in y["V"].columns:
    string = "df['"+x+"']"
    string.replace('\"','')
    a.append(string)

In [36]:
a

["df['Date']",
 "df['Open Price']",
 "df['High Price']",
 "df['Low Price']",
 "df['Close Price']",
 "df['Adj Close Price']",
 "df['Volume']",
 "df['Percent Change']",
 "df['Normalized Returns']"]

In [208]:
graphMACD_buy_sell(y,"ZM",5,15,10)

In [125]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(x):
    return x

interact(f, x = 5);
#interact(graphMACD_buy_sell, df=y, ticker="ZM", fast=12, slow=26, macd=9)

5

In [38]:
tickers = input("Please provide stock symbols separated by commas, max 5:") #aapl (Apple, inc.)

x, y = getStockData(tickers)

In [33]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=len(stock_names), cols=1)

count = 1

for x in stock_names:
    fig.add_trace(
        go.Scatter(x=stock_data[x]['Date'], y=stock_data[x]['Normalized Returns'], name=stock_names[x]),
        row= count, col=1)
        
    count +=1

fig.update_layout(title='Normalized Stock Returns of {}'.format(stock_names[x]))

fig.show()